## Importing Libraries

In [1]:
!pip install surprise
!pip install powerlaw
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import NormalPredictor, KNNBasic, KNNWithZScore, KNNWithMeans, KNNBaseline
from surprise import SVD, NMF
from sklearn.metrics import mean_squared_error
from tabulate import tabulate
from collections import defaultdict
import powerlaw

## Loading and Preprocessing MovieLens Data

In [2]:
def load_and_preprocess_movielens(movies_path, ratings_path, tags_path):
    movies = pd.read_csv(movies_path)
    ratings = pd.read_csv(ratings_path)
    tags = pd.read_csv(tags_path)

    merged_data = pd.merge(ratings, movies, on='movieId', how='left')
    merged_data = pd.merge(merged_data, tags, on=['movieId', 'userId'], how='left')

    reduced_data = merged_data.head(20000)

    processed_data = reduced_data[['userId', 'movieId', 'rating', 'title', 'genres', 'tag']]

    return processed_data

## Splitting Data

In [3]:
def split_data(data):
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    return train_data, test_data

## Defining Models

In [4]:
def define_models():

    models = {
        'Random': NormalPredictor(),
        'KNNBasic': KNNBasic(),
        'KNNWithZScore': KNNWithZScore(),
        'KNNWithMeans': KNNWithMeans(),
        'KNNBaseline': KNNBaseline(),
        'SVD': SVD(),
        'NMF': NMF(),
    }

    return models

In [5]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def bpr(predictions, targets):
    return np.mean(predictions)

def number_non_zeros(targets):
    return np.count_nonzero(targets)


## Defining Evaluation Metrics

In [6]:
def define_metrics():
    metrics = {
        'RMSE': rmse,
        'BPR': bpr,
        'NumberNonZeros': number_non_zeros,
    }
    return metrics

## Training Models

In [7]:
def train_model(model_name, train_data):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

    if model_name in ['Random', 'KNNBasic', 'KNNWithZScore', 'KNNWithMeans', 'KNNBaseline', 'SVD', 'NMF']:
        model = define_models()[model_name]
    else:
        raise ValueError(f"Unknown model: {model_name}")

    trainset, _ = surprise_train_test_split(data, test_size=0.2, random_state=42)
    model.fit(trainset)
    return model

## Evaluating Models

In [17]:
def evaluate_model(model, test_data, stratum=None):
    reader = Reader(rating_scale=(1, 5))
    test_data = Dataset.load_from_df(test_data[['userId', 'movieId', 'rating']], reader)
    test_data = test_data.build_full_trainset().build_testset()


    predictions = model.test(test_data)

    rmse = np.sqrt(mean_squared_error([pred.r_ui for pred in predictions], [pred.est for pred in predictions]))

    num_non_zeros = len(test_data)

    bpr = np.mean([pred.est for pred in predictions])

    results = {'rmse': rmse, 'number_non_zeros': num_non_zeros, 'bpr': bpr, 'stratum': stratum}

    return results

Stratified Evaluation

In [9]:
def estimate_propensities(data):

    # find the item's frequencies
    item_freq = defaultdict(int)
    for u, i, r in data:
        item_freq[i] += 1

    # fit the exponential param
    data = np.array([e for e in item_freq.values()], dtype=np.float)
    results = powerlaw.Fit(data, discrete=True, fit_method='Likelihood')
    alpha = results.power_law.alpha
    fmin = results.power_law.xmin

    # replace raw frequencies with the estimated propensities
    for k, v in item_freq.items():
        if v > fmin:
            item_freq[k] = pow(v, alpha)

    return item_freq  # user-independent propensity estimations

def build_stratified_datasets(train_data, test_data, n_strata):
    reader = Reader(rating_scale=(1, 5))
    test_data = Dataset.load_from_df(test_data[['userId', 'movieId', 'rating']], reader)
    train_data = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

    test_data = test_data.build_full_trainset().build_testset()

    train_data = train_data.build_full_trainset().build_testset()

    props = estimate_propensities(train_data)
    stratified_sets = {}
    test_props = np.array([props[i] for u, i, r in test_data], dtype=np.float64)
    strata, bins = pd.cut(x=test_props, bins=n_strata,labels=['Q%d' % i for i in range(1, n_strata+1)],retbins=True)
    for stratum in sorted(np.unique(strata)):
        # sample the corresponding sub-population
        qtest_data = []
        for (u, i, r), q in zip(test_data, strata):
            if q == stratum:
                qtest_data.append((u, i, r))
        stratified_sets[stratum] = qtest_data
    return stratified_sets

def stratified_evaluation(model, train_data, test_data, n_strata):
  stratified_sets = build_stratified_datasets(train_data, test_data, n_strata)
  results =[]
  for i in range(n_strata):
    label = 'Q%d' % (i + 1)
    predictions = model.test(stratified_sets[label])

    rmse = np.sqrt(mean_squared_error([pred.r_ui for pred in predictions], [pred.est for pred in predictions]))

    num_non_zeros = len(stratified_sets[label])

    bpr = np.mean([pred.est for pred in predictions])
    result = {'rmse': rmse, 'number_non_zeros': num_non_zeros, 'bpr': bpr, 'stratum': label}
    results.append(result)
  return results


## Comparing Correlations

In [10]:
def compare_correlations(correlation1, correlation2):
    test_statistic, p_value = ttest_rel(correlation1, correlation2)
    return test_statistic, p_value

## Verifying Simpson's Paradox

In [11]:
def verify_simpsons_paradox(results):
    overall_rmse = np.mean([result['rmse'] for result in results])
    stratified_rmse_q1 = np.mean([result['rmse'] for result in results if result['stratum'] == 1 and not np.isnan(result['rmse'])])
    stratified_rmse_q2 = np.mean([result['rmse'] for result in results if result['stratum'] == 2 and not np.isnan(result['rmse'])])

    print("Overall RMSE:", overall_rmse)
    print("Stratified RMSE (Q1):", stratified_rmse_q1)
    print("Stratified RMSE (Q2):", stratified_rmse_q2)

    if stratified_rmse_q1 > overall_rmse and stratified_rmse_q2 > overall_rmse:
        print("Simpson's Paradox is observed.")
    else:
        print("Simpson's Paradox is not observed.")


## Printing Metrics Table

In [12]:
def print_metrics_table(results):
    headers = ['Model', 'RMSE', 'BPR', 'Number of Non-Zeros','Stratum']

    table_data = []
    for result in results:
        row = [
            result.get('model', ''),
            result.get('rmse', ''),
            result.get('bpr', ''),
            result.get('number_non_zeros', ''),
            result.get('stratum', '')
        ]
        table_data.append(row)

    print(tabulate(table_data, headers=headers, tablefmt="grid"))


In [13]:
movies_path = 'movies.csv'
ratings_path = 'ratings.csv'
tags_path = 'tags.csv'
processed_data = load_and_preprocess_movielens(movies_path, ratings_path, tags_path)

print("Dataset Info:")
print(processed_data.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   20000 non-null  int64  
 1   movieId  20000 non-null  int64  
 2   rating   20000 non-null  float64
 3   title    20000 non-null  object 
 4   genres   20000 non-null  object 
 5   tag      612 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.1+ MB
None


In [14]:
train_data, test_data = train_test_split(processed_data, test_size=0.2, random_state=42)


In [15]:
models_to_evaluate = ['Random', 'KNNBasic', 'KNNWithZScore', 'KNNWithMeans', 'KNNBaseline', 'SVD', 'NMF']


In [18]:
results = []
for model_name in models_to_evaluate:
    trained_model = train_model(model_name, train_data)
    evaluation_results = evaluate_model(trained_model, test_data)

    if 'model' not in evaluation_results:
        evaluation_results['model'] = model_name
    stratified_evaluation_results = stratified_evaluation(trained_model, train_data, test_data, 2)
    results.append(evaluation_results)
    results += stratified_evaluation_results

<ipython-input-9-ee4a53dcc6bf>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array([e for e in item_freq.values()], dtype=np.float)


Calculating best minimal value for power law fit
Computing the msd similarity matrix...
Done computing similarity matrix.


<ipython-input-9-ee4a53dcc6bf>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array([e for e in item_freq.values()], dtype=np.float)


Calculating best minimal value for power law fit
Computing the msd similarity matrix...
Done computing similarity matrix.
Calculating best minimal value for power law fit


<ipython-input-9-ee4a53dcc6bf>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array([e for e in item_freq.values()], dtype=np.float)


...

<ipython-input-9-ee4a53dcc6bf>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array([e for e in item_freq.values()], dtype=np.float)


Calculating best minimal value for power law fit


<ipython-input-9-ee4a53dcc6bf>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array([e for e in item_freq.values()], dtype=np.float)


Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


<ipython-input-9-ee4a53dcc6bf>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array([e for e in item_freq.values()], dtype=np.float)


In [20]:
print_metrics_table(results)


+---------------+----------+---------+-----------------------+-----------+
| Model         |     RMSE |     BPR |   Number of Non-Zeros | Stratum   |
+===============+==========+=========+=======================+===========+
| Random        | 1.3807   | 3.58581 |                  4000 |           |
+---------------+----------+---------+-----------------------+-----------+
|               | 1.40368  | 3.58011 |                  3942 | Q1        |
+---------------+----------+---------+-----------------------+-----------+
|               | 1.37554  | 3.63272 |                    58 | Q2        |
+---------------+----------+---------+-----------------------+-----------+
| KNNBasic      | 1.02611  | 3.65234 |                  4000 |           |
+---------------+----------+---------+-----------------------+-----------+
|               | 1.02789  | 3.64346 |                  3942 | Q1        |
+---------------+----------+---------+-----------------------+-----------+
|               | 0.89727